<a href="https://colab.research.google.com/github/albertometelli/rl-phd-2020/01_intro_gym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# G(PO)MDP

This notebook is inspired to the Stable Baselines3 tutorial available at [https://github.com/araffin/rl-tutorial-jnrr19](https://github.com/araffin/rl-tutorial-jnrr19).


## Introduction

In this notebook, we will implement a very basic version of **G(PO)MDP**.

### Links

Open AI Gym Github: [https://github.com/openai/gym](https://github.com/openai/gym)

Open AI Gym Documentation: [https://www.gymlibrary.ml](https://www.gymlibrary.ml)

Stable Baselines 3 Github:[https://github.com/DLR-RM/stable-baselines3](https://github.com/DLR-RM/stable-baselines3)

Stable Baseline 3 Documentation: [https://stable-baselines3.readthedocs.io/en/master/](https://stable-baselines3.readthedocs.io/en/master/)

## Install Dependencies and Stable Baselines3 Using Pip

In [1]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common \
    python3-dev

!apt-get install -y patchelf
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
!pip install stable-baselines3[extra]
!pip install free-mujoco-py

Reason for being yanked: re-release with new wheels
  Using cached atari-py-0.2.6.tar.gz (790 kB)
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-z83_11xt/atari-py/setup.py'"'"'; __file__='"'"'/tmp/pip-install-z83_11xt/atari-py/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-ud50e_tf
       cwd: /tmp/pip-install-z83_11xt/atari-py/
  Complete output (502 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.8
  creating build/lib.linux-x86_64-3.8/atari_py
  copying atari_py/ale_python_interface.py -> build/lib.linux-x86_64-3.8/atari_py
  copying atari_py/__init__.py -> build/lib.linux-x86_64-3.8/atari_py
  copying atari_py/ale_c_wrapper.cpp -> build/lib.linux-x86_6

Failed to build atari-py
    Running setup.py install for atari-py ... error
    ERROR: Command errored out with exit status 1:
     command: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-z83_11xt/atari-py/setup.py'"'"'; __file__='"'"'/tmp/pip-install-z83_11xt/atari-py/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-eyz6l8rd/install-record.txt --single-version-externally-managed --user --prefix= --compile --install-headers /home/alberto/.local/include/python3.8/atari-py
         cwd: /tmp/pip-install-z83_11xt/atari-py/
    Complete output (504 lines):
    running install
    running build
    running build_py
    creating build
    creating build/lib.linux-x86_64-3.8
    creating build/lib.linux-x86_64-3.8/atari_py
    copying atari_py/ale_python_interface.py -> build/lib.linux-x86

ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-z83_11xt/atari-py/setup.py'"'"'; __file__='"'"'/tmp/pip-install-z83_11xt/atari-py/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-eyz6l8rd/install-record.txt --single-version-externally-managed --user --prefix= --compile --install-headers /home/alberto/.local/include/python3.8/atari-py Check the logs for full command output.


In [4]:
import stable_baselines3
stable_baselines3.__version__

'1.4.0'

## Evaluation

A helper function to evaluate policies.

In [5]:
def evaluate(env, policy, gamma=1., num_episodes=100):
    """
    Evaluate a RL agent
    :param env: (Env object) the Gym environment
    :param policy: (BasePolicy object) the policy in stable_baselines3
    :param gamma: (float) the discount factor
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    discounter = 1.
    all_episode_rewards = []
    for i in range(num_episodes): # iterate over the episodes
        episode_rewards = []
        done = False
        obs = env.reset()
        while not done: # iterate over the steps until termination
            action, _ = policy.predict(obs)
            obs, reward, done, info = env.step(action)
            episode_rewards.append(reward * discounter) # compute discounted reward
            discounter *= gamma

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    std_episode_reward = np.std(all_episode_rewards) / np.sqrt(num_episodes - 1)
    print("Mean reward:", mean_episode_reward, 
          "Std reward:", std_episode_reward,
          "Num episodes:", num_episodes)

    return mean_episode_reward, std_episode_reward

## Plotting

A helper function to plot the learning curves.

In [6]:
import matplotlib.pyplot as plt


def plot_results(results):
    plt.figure()
    
    for k in results.keys():
        data = np.load(results[k] + '/evaluations.npz')
        ts = data['timesteps']
        res = data['results']
        _mean, _std = res.mean(axis=1), res.std(axis=1)

        plt.plot(ts, _mean, label=k)
        plt.fill_between(ts, _mean-_std, _mean+_std, alpha=.2)
        
    plt.xlabel('Timesteps')
    plt.ylabel('Average return')
    plt.legend(loc='lower right')
    
    plt.show()

## G(PO)MDP

![ss](gpomdp.png)

**References**

Baxter, Jonathan, and Peter L. Bartlett. "Infinite-horizon policy-gradient estimation." Journal of Artificial Intelligence Research 15 (2001): 319-350.

## Policy

We will use a Gaussian policy, linear in the state variables and with fixed (non-learnable) standard deviation. 

$$
\pi_{\boldsymbol{\theta}}(a|\mathbf{s}) = \mathcal{N}(a| \boldsymbol{\theta}^T \mathbf{s}, \sigma^2)
$$

The policy must implement the usual `predict` method and some additional methods for computing the policy gradient. Specifically, we will need a `grad_log` method to return the gradient of the logarithm of the policy (the score):

$$
\nabla_{\boldsymbol{\theta}} \log \pi_{\boldsymbol{\theta}}(a|\mathbf{s})= \frac{(a - \boldsymbol{\theta}^T \mathbf{s})\mathbf{s}}{\sigma^2}
$$

In [7]:
class GaussianPolicy:
    
    def __init__(self, dim, std=0.1):
        """
        :param dim: number of state variables
        :param std: fixed standard deviation
        """
        
        self.std = std
        self.dim = dim
        self.theta = np.zeros((dim,))  # zero initializatoin
    
    def get_theta(self):
        return self.theta
    
    def set_theta(self, value):
        self.theta = value
    
    def predict(self, obs):
        mean = np.dot(obs, self.theta)
        action = mean + np.random.randn() * self.std
        return np.array([action]), obs
    
    def grad_log(self, obs, action):
        mean = np.dot(obs, self.theta)
        return (action - mean) * obs / self.std ** 2

## Training Routine

We provide the already implemented skeleton of the training routine that samples at every iterations $m$ trajectories from the environment.

In [8]:
def collect_rollouts(env, policy, m, T):
    """
    Collects m rollouts by running the policy in the
        environment
    :param env: (Env object) the Gym environment
    :param policy: (Policy object) the policy
    :param gamma: (float) the discount factor
    :param m: (int) number of episodes per iterations
    :param K: (int) maximum number of iterations
    :param theta0: (ndarray) initial parameters (d,)
    :param alpha: (float) the constant learning rate
    :param T: (int) the trajectory horizon
    :return: (list of lists) one list per episode
                each containing triples (s, a, r)
    """
    
    ll = []
    for j in range(m):
        s = env.reset()
        t = 0
        done = False
        l = []
        while t < T and not done:
            a, _ = policy.predict(s)
            s1, r, done, _ = env.step(a)
            l.append((s, a, r))
            s = s1
        ll.append(l)
    return ll
            
def train(env, policy, gamma, m, K, alpha, T):
    """
    Train a policy with G(PO)MDP
    :param env: (Env object) the Gym environment
    :param policy: (Policy object) the policy
    :param gamma: (float) the discount factor
    :param m: (int) number of episodes per iterations
    :param K: (int) maximum number of iterations
    :param alpha: (float) the constant learning rate
    :param T: (int) the trajectory horizon
    :return: None
    """
    
    for k in range(K):
        
        print('Iteration:', k)
        
        #Generate rollouts
        rollouts = collect_rollouts(env, policy, m, T)
        
        #Get policy parameter
        theta = policy.get_theta()
        print('\tParameters:', theta)
        
        #Call your G(PO)MDP estimator
        pg = gpomdp(rollouts, policy, gamma)
        
        # Update policy parameter
        theta = theta + alpha * pg
        
        #Set policy parameters
        policy.set_theta(theta)
        
        evaluate(env, policy, gamma)
        
        

The following is a possible implementation of G(PO)MDP that turns out to be very inefficient! It can be improved by precomputing the scores at the beginning and using matrix opertions instead of for cicles.

In [9]:
def gpomdp(rollouts, policy, gamma):
    
    grad = 0
    
    # Very very inefficient implementation!
    for roll in rollouts:
        H = len(roll)
        
        sum_rew = 0.
        for t in range(H):
            
            sum_scores = 0.
            for l in range(t + 1):
                s, a, _ = roll[l]
                score = policy.grad_log(s, a)
                sum_scores += score
            
            _, _, r = roll[t]
            sum_rew += gamma ** t * r * sum_scores
        
        grad += sum_rew
    
    return grad / len(rollouts)   

Let us test it!

In [10]:
import mujoco_py

running build_ext
building 'mujoco_py.cymj' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/home/alberto/.local/lib/python3.8/site-packages/mujoco_py -I/home/alberto/.local/lib/python3.8/site-packages/mujoco_py/binaries/linux/mujoco210/include -I/home/alberto/.local/lib/python3.8/site-packages/numpy/core/include -I/usr/include/python3.8 -c /home/alberto/.local/lib/python3.8/site-packages/mujoco_py/cymj.c -o /home/alberto/.local/lib/python3.8/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-3.8/home/alberto/.local/lib/python3.8/site-packages/mujoco_py/cymj.o -fopenmp -w
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=for

CompileError: command 'x86_64-linux-gnu-gcc' failed with exit status 1

In [ ]:
import gym
import numpy as np


#Instantiate the environment
env = gym.make('Pendulum-v0')

#Instantiate the policy
policy = GaussianPolicy(env.observation_space.shape[0], std=2)

#Other parameters
gamma = 0.99
m = 20
K = 20
alpha = 0.001
T = 100

train(env, policy, gamma, m, K, alpha, T)

Iteration: 0
	Parameters: [0. 0. 0.]
Mean reward: -7.859991506673208 Std reward: 6.7157023584383175 Num episodes: 100
Iteration: 1
	Parameters: [-0.18944941  0.26758589 -1.2382909 ]
Mean reward: -9.01409086678231 Std reward: 7.9114032928919045 Num episodes: 100
Iteration: 2
	Parameters: [-0.37598829 -0.06157708 -1.86202814]
Mean reward: -8.781609984884277 Std reward: 7.726040507408961 Num episodes: 100
Iteration: 3
	Parameters: [-0.89099668 -0.16507964 -1.52975174]
Mean reward: -9.00720260828952 Std reward: 7.939470785957481 Num episodes: 100
Iteration: 4
	Parameters: [-1.10388617 -0.28137463 -1.43747972]
Mean reward: -8.559273356910387 Std reward: 7.48193912459331 Num episodes: 100
Iteration: 5
	Parameters: [-1.98712175 -0.4373299  -1.68586476]
Mean reward: -7.940612329768299 Std reward: 6.892122410503793 Num episodes: 100
Iteration: 6
	Parameters: [-2.30793693 -0.55096147 -1.83268079]
Mean reward: -8.011813692885303 Std reward: 7.0128580593471295 Num episodes: 100
Iteration: 7
	Param

In [ ]:
env.observation_space.shape